# Download and extract dataset

In [ ]:
# Download dataset from Google Drive

import urllib.request

url = 'https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040'
filename = 'tennis_court_det_dataset.zip'
urllib.request.urlretrieve(url, filename)

In [ ]:
# Unzip dataset

import zipfile

with zipfile.ZipFile("tennis_court_det_dataset.zip", "r") as zip_ref:
    zip_ref.extractall("tennis_court_det_dataset")


# Start Code

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create Torch Dataset

In [12]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as file:
            self.data = json.load(file)
        
        # Transform images to standardize them
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    # Define length of dataset
    def __len__(self):
        return len(self.data)
    
    # Read items one by one and return the image and keypoints
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [13]:
train_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_train.json")
val_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

### Create model

In [14]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [15]:
model = model.to(device)

### Train model

In [16]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # Can modify learning rate

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14418.3203125
Epoch 0, iter 10, loss: 13766.9384765625
Epoch 0, iter 20, loss: 13910.9345703125
Epoch 0, iter 30, loss: 13963.431640625
Epoch 0, iter 40, loss: 13672.4736328125
Epoch 0, iter 50, loss: 13032.205078125
Epoch 0, iter 60, loss: 12857.3173828125
Epoch 0, iter 70, loss: 12166.8154296875
Epoch 0, iter 80, loss: 11946.3505859375
Epoch 0, iter 90, loss: 11678.685546875
Epoch 0, iter 100, loss: 11645.51953125
Epoch 0, iter 110, loss: 10885.87109375
Epoch 0, iter 120, loss: 10093.560546875
Epoch 0, iter 130, loss: 9761.4951171875
Epoch 0, iter 140, loss: 9180.22265625
Epoch 0, iter 150, loss: 9355.4638671875
Epoch 0, iter 160, loss: 9411.640625
Epoch 0, iter 170, loss: 8959.779296875
Epoch 0, iter 180, loss: 8060.12744140625
Epoch 0, iter 190, loss: 7894.27587890625
Epoch 0, iter 200, loss: 7641.1875
Epoch 0, iter 210, loss: 7097.3154296875
Epoch 0, iter 220, loss: 7289.990234375
Epoch 0, iter 230, loss: 6832.5986328125
Epoch 0, iter 240, loss: 6233.7783203

In [19]:
# Save predicted keypoints
torch.save(model.state_dict(), "keypoints_model.pth")